In [ ]:
%run params.ipynb

In [ ]:
if 'rules' in PARAMS:
    boolean_rules = PARAMS['rules']
else:
    import rules.example
    boolean_rules = rules.example.build_rules()['rules']
        
# boolean_rules

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pylab as pl
import boolean2 as bl
import pandas as pd
import numpy as np
from time import gmtime, strftime
import utils
import warnings

In [ ]:
model = bl.Model(boolean_rules, mode='sync')

print("Running boolean simulations")

for c in range(PARAMS["chunks"]):
    chunk = []
    for i in range(PARAMS["chunk_size"]):
        model.initialize(missing=bl.util.randbool)
        model.iterate(steps=30)

        dic = {}
        (cycle_start, cycle_len) = model.detect_cycles()
        dic['cycle_start'] = cycle_start
        dic['cycle_len'] = cycle_len
        for key, value in sorted(model.data.items()):    
            dic['init_'+key] = int(value[0])
            dic['cycle_'+key] = ''.join(map(lambda x: str(x*1), value[cycle_start:(cycle_start + cycle_len)]))
        dic['data'] = model.data    
        chunk.append(dic)

    warnings.filterwarnings('ignore')    
    pd.DataFrame(chunk).to_hdf(
        build_filename(PARAMS, "simulations.h5"),     
        key="%s" % strftime("chunk_%Y_%m_%d__%H_%M_%S", gmtime()))    
    warnings.filterwarnings('default')
    
    print("Saved chunk {}".format(c))